In [9]:
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from typing import Tuple
from cokonemliproje.modified_KMeans.modified_KMeans import modified_KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [22]:
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, ClusterMixin

In [4]:
def data_spliter(self,X:pd.DataFrame, y:pd.DataFrame, labels:np.array, index:int) -> Tuple[pd.DataFrame,pd.DataFrame]:
        """
        Args:
            X (pd.DataFrame): Feature input of dataframe.
            y (pd.DataFrame): Target input of dataframe.
            labels (np.array): Output of kmeans cluster.
            index (int): index in labels.

        Returns:
             Tuple[pd.DataFrame,pd.DataFrame] : X_loc and y_loc
        """        
        indexs = np.where(labels==index)
        X_loc = X.iloc[indexs]
        y_loc = y.iloc[indexs]
        
        return X_loc, y_loc

In [5]:
class BaseCP(BaseEstimator):
    def __init__(self,
                model:callable,
                selected_col = None,
                outlier_detection:bool = False,
                outlier_detection_cluster:bool = False) -> None:

        self.model = model
        self.selected_col = selected_col
        self.outlier_detection = outlier_detection
        self.outlier_detection_cluster = outlier_detection_cluster

    @staticmethod
    def __data_spliter(X:pd.DataFrame, y:pd.DataFrame, labels:np.array, index:int) -> Tuple[pd.DataFrame,pd.DataFrame]:
        """
        Args:
            X (pd.DataFrame): Feature input of dataframe.
            y (pd.DataFrame): Target input of dataframe.
            labels (np.array): Output of kmeans cluster.
            index (int): index in labels.

        Returns:
             Tuple[pd.DataFrame,pd.DataFrame] : X_loc and y_loc
        """        
        indexs = np.where(labels==index)
        X_loc = X.iloc[indexs]
        y_loc = y.iloc[indexs]
        
        return X_loc,y_loc

    @staticmethod
    def scaler(self,X1,X2=None):
        from sklearn.preprocessing import StandardScaler

        scaler = StandardScaler()
        X1 = scaler.fit_transform(X1)

        if X2 != None:
            X2 = scaler.transform(X2)
            return X1, X2
        else:
            return X1


In [8]:
class CP_Transformer(BaseCP,TransformerMixin):
    def __init__(self,selected_col=None):
        self.selected_col = selected_col

    def fit(self, X, y=None):
        return self
        
    def transform(self, X):
        X = X.loc[:,self.selected_col]
        return X

In [21]:
from sklearn import pipeline


class CPRegressor(BaseCP,RegressorMixin):
    def __init__(self, model: callable, n_clusters:int, scale=lambda: StandardScaler(), selected_col=None, cluster=lambda: KMeans()) -> None:
            self.model = model
            self.selected_col = selected_col
            self.n_clusters = n_clusters
            self.scale = True
            self.cluster = cluster

    def __cluster(self):
        if self.selected_col is not None:
            pipe = pipeline([CP_Transformer(selected_col=self.selected_col), self.cluster()])
        else:
            pipe = self.cluster()
        self.pipe = pipe

    def __cluster_fit(self,X,y):
        if "y" in self.pipe.fit.__code__.co_varnames:
            self.pipe.fit_transform(X,y)
        else:
            self.pipe.fit_transform(X)
        
    def __cluster_predict(self,X,y):
        if "y" in self.pipe.fit.__code__.co_varnames:
            predict = self.pipe.predict(X,y)
        else:
            predict = self.pipe.predict(X)

    

    def fit(self,X,y):

        self.clusters_ = {}
        self.fit_labels_ = {}



        

        def __sub_fit(X,y,cluster_:int,scale:bool):
            cluster, labels = self.__k_meanscluster(X,y,n_clusters=np.abs(cluster_))
            self.clusters_[cluster_] = cluster
            self.fit_labels_[cluster_] = labels
            std_dict = {}
            best_estimator_dict = {}
            mean_error_dict = {}
            shape_list = {}
            for index in range(np.abs(cluster_)):
                X_loc,y_loc = self.__data_spliter(X, y, labels, index)
                if X_loc.shape[0] < self.model.cv:
                    break
                if self.__scale == True:
                    X_loc = self.__scaler(X_loc,X2=None)
                best_estimator, std, mean_error =  self.__model_fit(X_loc, y_loc)
                best_estimator_dict[index+1] = best_estimator
                std_dict[index+1] = std
                mean_error_dict[index+1] = mean_error
                shape_list[index+1] = X_loc.shape[0]
            self.best_estimator_[cluster_] = best_estimator_dict
            self.cv_std_[cluster_] = std_dict
            self.cv_mean_error_[cluster_] = mean_error_dict
            self.fit_shape_[cluster_] = shape_list
            if self.model.verbose > 0:
                print(f"Cluster {cluster_} completed.")


    def fit(self,X,y):
        pass
    
    def predict(self,X):
        pass
